ターミナルに  
export HEADQUARTER_DATABASE_ID  
と  
export BOOKMARK_DATABASE_ID  
と  
NOTION_INTEGRATION_TOKEN  
をしてから実行してください。

GraphRAG準備

In [ ]:
!python -m graphrag.index --init --root ./ragtarget

settings.yamlのmodelをgpt-4o-miniに変更して安くします。  
あと、.env内のAPI_KEYを自分のOPENAI_API_KEYに変更すること。

In [29]:
import os
os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY')
os.environ['NOTION_INTEGRATION_TOKEN'] = os.getenv('NOTION_INTEGRATION_TOKEN')
os.environ["GEMINI_API_KEY"] = os.environ.get("GEMINI_API_KEY")
os.environ['HEADQUARTER_DATABASE_ID'] = os.environ.get('HEADQUARTER_DATABASE_ID')
os.environ['BOOKMARK_DATABASE_ID'] = os.environ.get('BOOKMARK_DATABASE_ID')
headquarter_database_id = os.environ.get("HEADQUARTER_DATABASE_ID")
BOOKMARK_DATABASE_ID = os.environ.get("BOOKMARK_DATABASE_ID")
integration_token = os.environ.get("NOTION_INTEGRATION_TOKEN")
print(integration_token)
print(headquarter_database_id)
print(BOOKMARK_DATABASE_ID)
# integration_token='うまくいかないので'
# HEADQUARTER_DATABASE_ID='直書きしてました'
# BOOKMARK_DATABASE_ID='すいません。'


TypeError: str expected, not NoneType

# NotionをGraphRAGで扱うための準備

In [11]:
import logging
import sys
from llama_index.core import SummaryIndex
from IPython.display import Markdown, display
import os
from tqdm import tqdm
import requests
from llama_index.readers.notion import NotionPageReader
import urllib3
# ログレベルの設定
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)
urllib3.disable_warnings()
logging.getLogger("urllib3").setLevel(logging.WARNING)

In [12]:
from llama_index.core import Settings, VectorStoreIndex
from llama_index.llms.openai import OpenAI
# from llama_index.llms.gemini import Gemini # ColabじゃないとGCPの設定が必要？
# from llama_index.embeddings.gemini import GeminiEmbedding # ColabじゃないとGCPの設定が必要？

# データ準備

In [19]:
# notion_test.py
def get_database_pages(databese_id):
    url = f"https://api.notion.com/v1/databases/{databese_id}/query"
    headers = {
        "Notion-Version": "2022-06-28",
        'Authorization': 'Bearer ' + integration_token,
        'Content-Type': 'application/json',
    }
    response = requests.post(url, headers=headers)
    return response.json()

def extract_page_titles(database_data):
    page_titles = []
    for page in database_data.get('results', []):
        properties = page.get('properties', {})
        for prop in properties.values():
            if prop['type'] == 'title':
                title = prop['title'][0]['plain_text'] if prop['title'] else ''
                page_titles.append(title)
                break
    return page_titles


# # 使用例
# database_data = get_database_pages(HEADQUARTER_DATABASE_ID)
# page_titles = extract_page_titles(database_data)
# print(page_titles)

def get_page_title(page_id):
    url = f"https://api.notion.com/v1/pages/{page_id}"
    headers = {
        "Notion-Version": "2022-06-28",
        'Authorization': 'Bearer ' + integration_token,
        'Content-Type': 'application/json',
    }
    response = requests.get(url, headers=headers)
    page_data = response.json()
    
    # ページのプロパティからタイトルを抽出
    properties = page_data.get('properties', {})
    for prop in properties.values():
        if prop['type'] == 'title':
            return prop['title'][0]['plain_text'] if prop['title'] else ''
    
    # タイトルが見つからない場合は空文字列を返す
    return ''

# 使用例
# page_id = "page_id here"
# title = get_page_title(page_id)
# print(f"Page Title: {title}")

In [20]:
# Initialize NotionPageReader
reader = NotionPageReader(integration_token=integration_token)

# Load data from Notion
# 指定したデータベースが持つページのデータを読み込む
headquarter_documents = reader.load_data(page_ids=reader.query_database(HEADQUARTER_DATABASE_ID))  # List of page IDs to load
bookmark_documents = reader.load_data(page_ids=reader.query_database(BOOKMARK_DATABASE_ID))  # List of page IDs to load

# 辞書型に整形
# ページタイトルをキー、ページ内容をバリューとする辞書を作成
docs = {}
documents = headquarter_documents + bookmark_documents
for doc in tqdm(documents):
    page_title, page_contents = get_page_title(doc.id_), doc.text
    docs[page_title] = page_contents

# 辞書をtxtに書き出す。キーをファイル名、値をコンテンツとして書き出す
for key, value in tqdm(docs.items()):
    file_name = key.replace('.', '_').replace('/', '-')
    with open(f"ragtarget/input/{file_name}.txt", "w") as f:
        f.write(value)

100%|██████████| 43/43 [00:00<00:00, 1949.41it/s]


# GraphRAG

GraphRAG用にグラフを作成

In [21]:
!python -m graphrag.index --root ./ragtarget

🚀 Reading settings from ragtarget/settings.yaml
⠙ GraphRAG Indexer 
⠙ GraphRAG Indexer e.text) - 43 files loaded (0 filtered) ━ 100%  0…
├── Loading Input (InputFileType.text) - 43 files loaded (0 filtered) ━ 100%  0…
⠙ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 43 files loaded (0 filtered) ━ 100%  0…
⠙ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 43 files loaded (0 filtered) ━ 100%  0…
⠙ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 43 files loaded (0 filtered) ━ 100%  0…
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 43 files loaded (0 filtered) ━ 100%  0…
└── create_base_text_units
⠸ GraphRAG Indexer ━━━━━━━━━━━━━━━━━━━━━━━━━━   2% -:--:-- 0:00:00
├── Loading Input (InputFileType.text) - 43 files loaded (0 filtered) ━ 100%  0…
⠸ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 43 files loaded (0 filtered) ━ 100%  0…
⠸ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 43 files loaded (0 filtered) ━ 100%  0…
⠸

In [16]:
!python -m graphrag.query \
--root ./ragtarget \
--method global \
"SQLを勉強するのに良い資料は？"



INFO: Reading settings from ragtarget/settings.yaml
creating llm client with {'api_key': 'REDACTED,len=51', 'type': "openai_chat", 'model': 'gpt-4o-mini', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': None, 'api_version': None, 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': None, 'model_supports_json': True, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Global Search Response: ## SQL学習のためのリソース

SQLを学ぶための優れた資料として、以下のポイントが挙げられます。

### SQL Tutorial

SQL Tutorialは、SQLとそのリレーショナルデータベースにおける応用を学ぶための主要な教育イベントです。このイベントでは、参加者がリアルタイムでSQLスキルを実践できるインタラクティブな演習が含まれており、技術スキルを向上させるための重要なリソースとなります [Data: Reports (41)]。

### SQLZOOプラットフォーム

SQLZOOは、SQL Tutorialをホストするオンラインプラットフォームであり、参加者がイベント後も学び続けるための追加リソースを提供しています。このコラボレーションにより、SQL教育のリーチとアクセス可能性が向上し、学習者にとって価値あるリソースとなっています [Data:

In [17]:
!python -m graphrag.query \
--root ./ragtarget \
--method local \
"SQLを勉強するのに良い資料は？"



INFO: Reading settings from ragtarget/settings.yaml
[2024-08-01T17:32:39Z WARN  lance::dataset] No existing dataset at /Users/mototsuguohki/Desktop/Work/AIVALIX/GraphRAG_NotionQA/lancedb/description_embedding.lance, it will be created
creating llm client with {'api_key': 'REDACTED,len=51', 'type': "openai_chat", 'model': 'gpt-4o-mini', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': None, 'api_version': None, 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': None, 'model_supports_json': True, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}
creating embedding llm client with {'api_key': 'REDACTED,len=51', 'type': "openai_embedding", 'model': 'text-embedding-3-small', 'max_tokens': 4000, 'temperature': 0, 'top_p': 1, 'n': 1, 'request_timeout': 180.0, 'api_base': None, 'api_versi